In [ ]:
!pip install dash
!pip install dash-bootstrap-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 2.0 MB/s eta 0:00:00


In [ ]:
from dash import Dash, Input, Output
from dash import html, dcc
import dash_bootstrap_components as dbc
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from dash import dash_table
import plotly.graph_objects as go

In [ ]:
xlsx_path = './data.xlsx'

# Read the CSV file into a DataFrame
df = pd.read_excel(xlsx_path)


In [ ]:
df = df.sort_values(by='Date')

In [ ]:
df['Libellé'].unique()

array(['AB SCIENCE', 'ABC ARBITRAGE', 'ACCOR', 'ADOCIA', 'ADP',
       'AIR FRANCE - KLM', 'AIR LIQUIDE', 'AIRBUS', 'AKWEL', 'ALSTOM',
       'ALTAMIR', 'ALTAREA', 'ALTEN', 'AMUNDI', 'ARGAN', 'ARKEMA',
       'ARTMARKET.COM', 'ASSYSTEM', 'ATOS', 'AUBAY', 'AVENIR TELECOM',
       'AXA', 'AXWAY SOFTWARE', 'AYVENS (ex ALD)', 'BAINS DE MER MONACO',
       'BELIEVE', 'BENETEAU', 'BIC', 'BIGBEN INTERACTIVE', 'BIOMERIEUX',
       'BNP PARIBAS', 'BOIRON', 'BOLLORE SE', 'BONDUELLE', 'BOUYGUES',
       'BUREAU VERITAS', 'CAPGEMINI', 'CARMILA', 'CARREFOUR',
       'CASINO GUICHARD PERRACHON', 'CATANA GROUP', 'CEGEDIM',
       'CHARGEURS', 'CHRISTIAN DIOR', 'CIE DES ALPES',
       'CIS (CATERING INTL SERVICES)', 'CLARANOVA', 'CLARIANE', 'COFACE',
       "COMP DE L'ODET", 'COVIVIO', 'CREDIT AGRICOLE SA', 'CRIT',
       'DANONE', 'DASSAULT AVIATION', 'DASSAULT SYSTEMES',
       'DBV TECHNOLOGIES', 'DMS', 'EDENRED', 'EIFFAGE', 'EKINOPS',
       'ELIOR GROUP', 'ELIS', 'EMEIS (EX ORPEA)', 'ENGIE', 'EQU

In [ ]:
home_layout = html.Div(
    children=[
        html.H1(children="This is what data look like"),
              dash_table.DataTable(
              id='datatable',
              columns=[{'name': col, 'id': col} for col in df.columns],
              data=df.to_dict('records'))
    ]
)


data_info_layout = html.Div(
    children=[html.H1(children="Here you will find some informations about the data"),
              html.Div([
              dash_table.DataTable(
                id='InfoTable',
                columns = [{'name': col, 'id': col} for col in df.describe().columns],
                data=df.describe().to_dict('records')
              ), html.H1(f'The shape of the data is'), html.P(str(df.shape))])
             ]

)

droped_df = df.drop(['Libellé', 'Date'], axis=1)

data_hist = html.Div(
    children = [
        html.H1("Here you will find some corelations between the variables"),
        html.Div([
        dcc.Dropdown(
            id='dataset-dropdown',
            options=[
                {'label': 'Close', 'value': 'Adj Close'},
                {'label': 'Volume', 'value': 'Volume'},
                {'label': 'Open', 'value': 'Open'},
                {'label': 'High', 'value': 'High'},
            ],
            value='data1'  # Default selected value
        )
    ]),

        dcc.Graph(
        id='histogram',
        figure={
            'data': [go.Histogram(x=df['Adj Close'])],
            'layout': go.Layout(title='Histogram')
        }
        )
    ]
)

companies = [df[df["Libellé"] == 'BTC'], df[df["Libellé"] == 'Apple Inc.'], df[df["Libellé"] == 'Microsoft Corporation'], df[df["Libellé"] == 'NIKE, Inc.']]

data_graph = html.Div(
    children = [
        html.H1("Here you will find some graph of closing values of some companies"),
        html.Div([
        dcc.Dropdown(
            id='dataset-dropdown2',
            options=[
                {'label': 'BTC', 'value': '0'},
                {'label': 'APPLE', 'value': '1'},
                {'label': 'MICROSOFT', 'value': '2'},
                {'label': 'NIKE', 'value': '3'},
            ],
            value='data1'  # Default selected value
        )
    ]),
        dcc.Graph(
        id='line-plot',
        figure={
            'data': [go.Scatter(x=companies[0]['Date'], y=companies[0]['Adj Close'])],
            'layout': go.Layout(title='Line Plot')
        }
    )
    ]
)


app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Home", href="/")),
        dbc.NavItem(dbc.NavLink("Informations", href="/data_information")),
        dbc.NavItem(dbc.NavLink("Histograms", href="/Histograms")),
        dbc.NavItem(dbc.NavLink("Graphs", href="/Graphs")),
    ],
    brand="DashBoard For Stock Market Anlayse",
    color="dark",
    dark=True,
    className="mb-2",
)

app.layout = html.Div(
    [
        dcc.Location(id="url", refresh=False),
        navbar,
        dbc.Container(id="page-content", className="mb-4", fluid=True),
    ]
)



@app.callback(
    Output('line-plot', 'figure'),
    [Input('dataset-dropdown2', 'value')]
)
def update_graph(i):
    return go.Figure(
        data=[go.Scatter(x=companies[int(i)]['Date'], y=companies[int(i)]['Adj Close'])],
        layout=go.Layout(title='Line Plot'))

@app.callback(
    Output('histogram', 'figure'),
    [Input('dataset-dropdown', 'value')]
)
def update_histogram(selected_dataset):
    return go.Figure(
        data=[go.Histogram(x=df[selected_dataset])],
        layout=go.Layout(title=selected_dataset))

@app.callback(Output("page-content", "children"), Input("url", "pathname"))
def display_page(pathname):
    if pathname == "/":
        return home_layout
    elif pathname == "/data_information":
        return data_info_layout
    elif pathname == "/Histograms":
        return data_hist
    elif pathname == "/Graphs":
        return data_graph
    else:
        return dbc.Jumbotron(
            [
                html.H1("404: Not found", className="text-danger"),
                html.Hr(),
                html.P(f"The pathname {pathname} was not recognized..."),
            ]
        )


if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>